In [ ]:
# default_exp config

# Config

> This module manages configuration of the `planetarypy` package.

In [ ]:
# hide
from nbverbose.showdoc import show_doc

In [ ]:
# export
import os
import shutil
from functools import reduce
from importlib.resources import path as resource_path
from typing import Union

import tomlkit as toml
from fastcore.utils import AttrDict, Path, dict2obj

In [ ]:
# exports
class Config:
    """Manage config stuff.

    Attributes
    -------
    path: pathlib.Path

    The key, value pairs found in the config file become attributes of the
    class instance after initialization.
    At minimum, there should be the `storage_root` attribute for storing data
    for this package.
    """

    # This part enables a config path location override using env PYCISS_CONFIG
    fname = "planetarypy_config.toml"
    # separating fname from fpath so that resource_path below is correct.
    path = Path(os.getenv("PLANETARYPY_CONFIG", Path.home() / f".{fname}"))

    def __init__(
        self, 
        config_path:str=None  # str or pathlib.Path
    ):
        "Switch to other config file location with `config_path`."
        if config_path is not None:
            self.path = Path(config_path)
        if not self.path.exists():
            with resource_path("planetarypy.data", self.fname) as p:
                shutil.copy(p, self.path)
        self.read_config()
        self.update_missions()

    def read_config(self):
        """Read the configfile and store config dict.

        `storage_root` will be stored as attribute.
        """
        self.tomldoc = toml.loads(self.path.read_text())
        if not self.tomldoc["storage_root"]:
            self.ask_storage_root()
        else:
            self.storage_root = Path(self.tomldoc["storage_root"])

    @property
    def d(self):
        "get the Python dic from"
        return self.tomldoc

    def get_value(
        self,
        key:str  # A nested key in dotted format, e.g. cassini.uvis.indexes
    ):
        """Get sub-dictionary by nested key."""
        if not key.startswith('missions'):
            key = 'missions.' + key
        try:
            return reduce(lambda c, k: c[k], key.split("."), self.d)
        except toml.exceptions.NonExistentKey:
            return None

    def set_value(
        self,
        nested_key:str,  # A nested key in dotted format, e.g. cassini.uvis.ring_summary 
        value:Union[float, str],  # Value for the given key to be stored
        save:bool=True  # Switch to control writing out to disk
    ):
        "Set value in sub-dic using dotted key."
        dic = self.tomldoc
        keys = nested_key.split(".")
        for key in keys[:-1]:
            dic = dic[key]
        dic[keys[-1]] = value
        if save:
            self.save()

    def save(self):
        "Write the TOML doc to file."
        self.path.write_text(toml.dumps(self.tomldoc))

    def ask_storage_root(self):
        """Use input() to ask user for the storage_root path.

        The path will be stored in the TOML-dict and saved into existing config file
        at `Class.path`, either default or as given during init.
        `storage_root` attribute is set as well.
        """
        path = input(
            "Provide the root storage path where all downloaded and produced data will be stored:"
        )
        self.tomldoc["storage_root"] = path
        self.storage_root = Path(path)
        self.save()

    @property
    def missions(self):
        return list(self.d["missions"].keys())

    def list_instruments(self, mission):
        if not mission.startswith("missions"):
            mission = "missions." + mission
        instruments = self.get_value(mission)
        return list(instruments.keys())

    def list_indexes(self, instrument):
        "instrument key needs to be <mission>.<instrument>"
        if not instrument.startswith("missions"):
            instrument = "missions." + instrument
        indexes = self.get_value(instrument + ".indexes")
        return list(indexes)

    def _copy_clean_to_resource(self):
        "Copy a clean config file without timestamps into resource path for repo commit."
        dic = self.tomldoc.copy()
        missions = dic["missions"]
        for mission in missions.keys():
            mdict = missions[mission]
            for instr in mdict.keys():
                instrdict = mdict[instr]
                for index in instrdict["indexes"]:
                    instrdict["indexes"][index]["timestamp"] = ""
        with resource_path("planetarypy.data", self.fname) as p:
            Path(p).write_text(toml.dumps(dic))

    def update_missions(self):
        "Check if a new version with more URLs exist at resource path."
        with resource_path("planetarypy.data", self.fname) as p:
            new = toml.loads(Path(p).read_text())["missions"]
        old = self.tomldoc["missions"]
        for mission in new:
            missiondata = new[mission]
            if mission not in old:
                old[mission] = missiondata
                continue
            for instr in missiondata:
                instrdata = missiondata[instr]
                if instr not in old[mission]:
                    old[mission][instr] = instrdata
                    continue
                for index in instrdata["indexes"]:
                    indexdata = instrdata["indexes"][index]
                    if index not in old[mission][instr]["indexes"]:
                        old[mission][instr]["indexes"][index] = indexdata
                        continue
                    oldindexdata = old[mission][instr]["indexes"][index]
                    if indexdata["url"] != oldindexdata["url"]:
                        oldindexdata["url"] = indexdata["url"]
        self.save()

    def populate_timestamps(self):
        pass

    def __repr__(self):
        return AttrDict(self.d).__repr__()

In [ ]:
# export
config = Config()

In [ ]:
config

- storage_root: /home/maye/big_drive/planetary_data
- missions: 
  - cassini: 
    - iss: 
      - indexes: 
        - index: 
          - url: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_index.lbl
        - inventory: 
          - url: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_inventory.lbl
        - moon_summary: 
          - url: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl
          - timestamp: 2021-07-23T22:11:38.904571
        - ring_summary: 
          - url: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_ring_summary.lbl
          - timestamp: 2021-07-01T09:47:46.753035
        - saturn_summary: 
          - url: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_saturn_summary.lbl
    - uvis: 
      - indexes: 
        - index: 
          - url: https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx

In [ ]:
for mission in missions.keys():
    mdict = missions[mission]
    for instr in mdict.keys():
        instrdict = mdict[instr]
        for index in instrdict["indexes"]:
            instrdict["indexes"][index]["timestamp"] = "test"

In [ ]:
for mission in config.list_missions():
    for instrument in config.list_instruments(mission):
        for index in config.list_indexes(f"{mission}.{instrument}"):
            #             key = ".".join(['missions', mission, instrument, index])
            print(index)

index
inventory
moon_summary
ring_summary
saturn_summary
index
moon_summary
ring_summary
saturn_summary
supplemental_index
versions
dtm
edr
rdr
edr
edr1
edr2
rdr1
rdr2
edr
rdr


## The Config() object

The `config` module instantiates a `config` object from the Config class.
Its attributes can be used to access several aspects relevant to the configuration of `planetarypy`.
Using an object approach enables easy growth and nesting over time.

> Note: Any cell that starts with an `# export` becomes part of the library.
  Any other cells become automatically part of tests.
  `exports` also adds the exported code to the docs.
  
> Note: A good "First issue" for collaborators would be to improve the Config class to be able to merge a new larger index catalog with an existing one the user might have configured. This will need to take care of existing timestamps and possible other indexes that were put in by the user.

In [ ]:
config = Config(config_path="test_config.toml")

In [ ]:
config.storage_root

Path('/home/maye/big_drive/planetary_data')

In [ ]:
# export
config = Config()

In [ ]:
config.storage_root

Path('/home/maye/big_drive/planetary_data')

In [ ]:
show_doc(Config.read_config)

<h4 id="Config.read_config" class="doc_header"><code>Config.read_config</code><a href="__main__.py#L30" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.read_config</code>()

```
Read the configfile and store config dict.

`storage_root` will be stored as attribute.
```

In [ ]:
show_doc(Config.get_value)

<h4 id="Config.get_value" class="doc_header"><code>Config.get_value</code><a href="__main__.py#L46" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.get_value</code>(**`key`**)

```
Get sub-dictionary by nested key.

Parameters
----------
nested_key: str
    A nested key in dotted format, e.g. cassini.uvis.indexes
```

In [ ]:
index = "missions.cassini.iss.indexes.moon_summary"

In [ ]:
config.get_value(index)

{'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl'}

In [ ]:
show_doc(Config.set_value)

<h4 id="Config.set_value" class="doc_header"><code>Config.set_value</code><a href="__main__.py#L59" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.set_value</code>(**`nested_key`**, **`value`**, **`save`**=*`True`*)

```
Set sub-dic using dotted key.

Parameters
----------
key: str
    A nested key in dotted format, e.g. cassini.uvis.ring_summary
value: convertable to string
    Value for the given key to be stored.
```

In [ ]:
show_doc(Config.save)

<h4 id="Config.save" class="doc_header"><code>Config.save</code><a href="__main__.py#L74" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.save</code>()

```
Write the TOML doc to file.
```

In [ ]:
config.get_value(index)

{'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl'}

In [ ]:
show_doc(Config.ask_storage_root)

<h4 id="Config.ask_storage_root" class="doc_header"><code>Config.ask_storage_root</code><a href="__main__.py#L78" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.ask_storage_root</code>()

```
Use input() to ask user for the storage_root path.

The path will be stored in the YAML-dict and saved into existing config file
at `Class.path`, either default or as given during init.
`storage_root` attribute is set as well.
```

In [ ]:
config.missions

['cassini', 'mro', 'lro']

In [ ]:
config.fname

'planetarypy_config.toml'

In [ ]:
config.path

Path('/home/maye/.planetarypy_config.toml')

In [ ]:
config.list_instruments("cassini")

['iss', 'uvis']

In [ ]:
config.list_indexes("cassini.iss")

['index', 'inventory', 'moon_summary', 'ring_summary', 'saturn_summary']

In [ ]:
config.d

{'storage_root': '/home/maye/big_drive/planetary_data', 'missions': {'cassini': {'iss': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_index.lbl'}, 'inventory': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_inventory.lbl'}, 'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl'}, 'ring_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_ring_summary.lbl'}, 'saturn_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_saturn_summary.lbl'}}}, 'uvis': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx/COUVIS_0999/COUVIS_0999_index.lbl'}, 'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx/COUVIS_0999/COUVIS_0999_moon_summary.lbl'}, 'ring_summary': {'url': 'https://pds-ri